In [68]:
import os
import json
import urllib
import requests
import random
from collections import OrderedDict
from IPython.display import display, HTML
from langchain.prompts import PromptTemplate
from langchain.chat_models import AzureChatOpenAI
from langchain.docstore.document import Document
from langchain.chains.question_answering import load_qa_chain
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from configparser import ConfigParser

parser = ConfigParser()
parser.read('../secrets.cfg')

AZURE_SEARCH_API_VERSION = parser.get('my_api','AZURE_SEARCH_API_VERSION')
AZURE_OPENAI_API_VERSION = parser.get('my_api','AZURE_OPENAI_API_VERSION')
AZURE_SEARCH_ENDPOINT = parser.get('my_api','AZURE_SEARCH_ENDPOINT')
AZURE_SEARCH_KEY = parser.get('my_api','AZURE_SEARCH_KEY')
AZURE_OPENAI_ENDPOINT = parser.get('my_api','AZURE_OPENAI_ENDPOINT')
AZURE_OPENAI_API_KEY = parser.get('my_api','AZURE_OPENAI_API_KEY')
PORTAL_TOKEN = parser.get('my_api','PORTAL_TOKEN')
URL  = parser.get('my_api','ASH_EXAMPLE_DATA_SOURCE')
MODEL = "gpt-35-turbo" # options: gpt-35-turbo, gpt-4, gpt-4-32k

# Setup the Payloads header
headers = {'Content-Type': 'application/json','api-key': AZURE_SEARCH_KEY}
ASHheaders = {"Authorization": PORTAL_TOKEN}
indexes = ["servicehealthfhl-index1"]

# Case 1: Summarization
Simple functionality that can generate a list of summaries at load time for the user to fetch




get data(using the api for fhl):

In [69]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

def fetch_data():
    # api-endpoint
    URL  = parser.get('my_api','ASH_EXAMPLE_DATA_SOURCE') 

    r = requests.get(url = URL, headers = ASHheaders)
        # extracting data in json format
    data = r.json()

    print(data)

    result = dict()

    if 'value' in data:
        for event in data['value']:           
            
            if event['name'][0] == '_' or 'properties' not in event:
                continue
            
            props = set (['title','impactStartTime','impactMitigationTime','description','impact'])

            if props.issubset(event['properties'].keys()) and event['properties']['title'][0] != '_':  
                propsDict = dict()
                propsDict['title'] = event['properties']['title']
                propsDict['chunks'] = [event['properties']['description']]# todo: implement actual chunking
                propsDict['language'] = 'en'
                propsDict['caption'] = event['properties']['title']
                propsDict['score']  = 3 
                propsDict['start time'] =  event['properties']['impactStartTime']
                propsDict['end time'] =  event['properties']['impactMitigationTime']

                result[event['name']] = propsDict

    return result                        
                
summary_data = fetch_data()


{'value': [{'id': 'Microsoft.ResourceHealth/events/BL8Y-DT8', 'name': 'BL8Y-DT8', 'type': 'Microsoft.ResourceHealth/events', 'properties': {'eventType': 'ServiceIssue', 'eventSource': 'ServiceHealth', 'status': 'Active', 'title': 'Azure Multi-Factor Authentication (MFA) - Applying Mitigation', 'summary': '<p><strong>Summary of impact:&nbsp;</strong>Between 05:38 UTC on 12 May 2023 and and 07:48 UTC on 20 May 2023, you were identified as a customer using Azure Multi-Factor Authentication (MFA) wherein users may be blocked while using Windows first time setup (Out of Box Experience) or Windows Intune AutoPilot and Authenticator App with Code Match is used as an MFA method.</p>', 'description': '<p><strong>Summary of impact:&nbsp;</strong>Between 05:38 UTC on 12 May 2023 and and 07:48 UTC on 20 May 2023, you were identified as a customer using Azure Multi-Factor Authentication (MFA) wherein users may be blocked while using Windows first time setup (Out of Box Experience) or Windows Intune

create langchain documents and get the most recent documents that are within the token limit

In [70]:
def create_langchain_documents(ordered_content):# Iterate over each of the results chunks and create a LangChain Document class to use further in the pipeline
    docs = []
    for key,value in ordered_content.items():
        for page in value["chunks"]:
            docs.append(Document(page_content=page, metadata={"source": key}))
    return docs
summary_docs  = create_langchain_documents(summary_data)

In [71]:
# functions for calculating token lengths
import tiktoken
from typing import List
# Returning the toekn limit based on model selection
def model_tokens_limit(model: str) -> int:
    """Returns the number of tokens limits in a text model."""
    if model == "gpt-35-turbo":
        token_limit = 3000
    elif model == "gpt-4":
        token_limit = 7000
    elif model == "gpt-4-32k":
        token_limit = 31000
    else:
        token_limit = 3000
    return token_limit

# Returns the num of tokens used on a string
def num_tokens_from_string(string: str) -> int:
    encoding_name ='cl100k_base'
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

def num_tokens_from_docs(docs: List[Document]) -> int:
    num_tokens = 0
    for i in range(len(docs)):
        num_tokens += num_tokens_from_string(docs[i].page_content)
    return num_tokens




In [72]:

def limit_docs_to_max_token_lenght(summary_docs):
    # Calculate number of tokens of our docs
    summary_docs_used = []
    summary_docs_used_token_count = 0
    for doc1 in summary_docs:
        summary_num_tokens = num_tokens_from_string(doc1.page_content)

        if summary_docs_used_token_count + summary_num_tokens < model_tokens_limit(MODEL):
            summary_docs_used_token_count = summary_docs_used_token_count + summary_num_tokens
            summary_docs_used.append(doc1)  

    print("Custom token limit for", MODEL, ":", model_tokens_limit(MODEL))
    #print("total number of documents used: ", len(all_docs_used))
    print("Summary docs count: ", len(summary_docs_used))
    print("Summary docs token count: ", num_tokens_from_docs(summary_docs))
    #print("Comparison docs token count: ", len(comparison_docs))
    return summary_docs_used
summary_docs_used = limit_docs_to_max_token_lenght(summary_docs)


Custom token limit for gpt-35-turbo : 3000
Summary docs count:  8
Summary docs token count:  18038


initialize session with llm:

In [73]:
# Set the ENV variables that Langchain needs to connect to Azure OpenAI
os.environ["OPENAI_API_BASE"] = os.environ["AZURE_OPENAI_ENDPOINT"] = AZURE_OPENAI_ENDPOINT
os.environ["OPENAI_API_KEY"] = os.environ["AZURE_OPENAI_API_KEY"] = AZURE_OPENAI_API_KEY
os.environ["OPENAI_API_VERSION"] = os.environ["AZURE_OPENAI_API_VERSION"] = AZURE_OPENAI_API_VERSION
os.environ["OPENAI_API_TYPE"] = "azure"  

# Create our LLM model
# Make sure you have the deployment named "gpt-35-turbo" for the model "gpt-35-turbo (0301)". 
# Use "gpt-4" if you have it available.

llm1 = AzureChatOpenAI(deployment_name=MODEL, temperature=0, max_tokens=500)



using a prompt and the above llm to generate a summary:

In [74]:
 
# this is very temporary
SUMMARY_COMBINE_PROMPT_TEMPLATE = """
These are examples of how you must provide the answer:
--> Beginning of examples
=========
Content: What happened?\n on 11 April 2023 Mat threw a party\nWhat are the Next steps?\nMat will throw more parties\nHow can customers make incidents like this less impactful?\nbuy headphones
Source: A12h2h4j5
Content: What happened?\n on 12 April 2023 Abhay ate an apple\nWhat are the Next steps?\nAbhay will sleep
Source: A12h2h4j5
Content: What happened?\non 13 April 2023 An elephant broke a wall\nHow will we fix the isse?\nwe will continue to investigate 
Source: j5j6k6
=========
SUMMARY: On 11 April 2023, Mat threw a party, and the next steps involve Mat planning to host more parties. To make incidents like this less impactful, customers are advised to buy headphones.On 12 April 2023, Abhay ate an apple, and the next step for Abhay is to sleep.On 13 April 2023, an elephant broke a wall. To address this issue, further investigation will be conducted.
SOURCES: A12h2h4j5, j5j6k6
=========
<-- End of examples
Given the following documents create a list of summaries. 
combine similar events together.  
Add important details from each document. Do not add details not present in the documents.
ALWAYS return a "SOURCES" part in your answer.
Instead of using the word "Document" use "Tracking ID"
Respond in {language}.
=========
{summaries}
=========
FINAL ANSWER IN {language}:
"""

SUMMARY_COMBINE_SUMMARY_PROMPT = PromptTemplate(
    template=SUMMARY_COMBINE_PROMPT_TEMPLATE, input_variables=["summaries", "language"]
)

chain1 = load_qa_with_sources_chain(llm1, chain_type="stuff", 
                                       prompt=SUMMARY_COMBINE_SUMMARY_PROMPT)

response1 = chain1({"input_documents": summary_docs_used,"question":"", "language": "English"})

#response1 = chain1({"input_documents": summary_docs,"question":"", "token_length": int(model_tokens_limit(MODEL)/2), "language": "English"})
answer1 = response1['output_text']
print(response1)
print(answer1)



{'input_documents': [Document(page_content='<p><strong>Summary of impact:&nbsp;</strong>Between 05:38 UTC on 12 May 2023 and and 07:48 UTC on 20 May 2023, you were identified as a customer using Azure Multi-Factor Authentication (MFA) wherein users may be blocked while using Windows first time setup (Out of Box Experience) or Windows Intune AutoPilot and Authenticator App with Code Match is used as an MFA method.</p>\n<p><br></p>\n<p><strong>Preliminary root cause:</strong>&nbsp;A recent code change to update the service introduced a bug that caused the issue.&nbsp;</p>\n<p><br></p>\n<p><strong>Mitigation:</strong>&nbsp;We rolled out a fix in the update code to mitigate the issue.&nbsp;</p>\n<p>&nbsp;</p>\n<p><strong>Next steps:</strong>&nbsp;We will continue to investigate to establish the full root cause and prevent future occurrences. Stay informed about Azure service issues by creating custom service health alerts:&nbsp;&nbsp;<a href="https://aka.ms/ash-videos" target="_blank">http

In [75]:
display(HTML('<h4>Azure OpenAI ChatGPT Answer:</h4>'))
display(HTML(answer1.split("SOURCES:")[0]))

# Case 2: Search    
The first Question the user asks:

In [76]:
QUESTION = "what are the Authentication issues?" # the question asked by the user


#### Lets find what events might be associated with this ask. We do this via keyword cognitive search and by in limited in memory vector search.
      The following sections are currently doing keyword search via cognitive search:
        if too many results retuned: then doing an in memory vector on these results to get top X.          
        if 0 results retuned: search on a small subset of the most recent data via a in memory vector search on the latest few tracking ids. Allow user to select next(not implemented) 
      This should all be replaced by doing a vector search using the in private preview version of cognitive search which has the capabilities of vector search baked in

The keyword search: 

In [77]:

def get_agg_search_results(skip = 0): # get the events the question might pertain to. currently gets 5 events in the example
    agg_search_results = [] 

    _skip = 5*skip
    for index in indexes:
        url = AZURE_SEARCH_ENDPOINT + '/indexes/'+ index + '/docs'
        url += '?api-version={}'.format(AZURE_SEARCH_API_VERSION)
        url += '&search={}'.format(QUESTION)
        url += '&select=*'
        url += '&$top=5'  # You can change this to anything you need/want
        url += '&queryLanguage=en-us'
        url += '&queryType=semantic'
        url += '&semanticConfiguration=servicehealthfhl-semantic-config'
        url += '&$count=true'
        url += '&speller=lexicon'
        url += '&answers=extractive|count-3'
        url += '&captions=extractive|highlight-false'
        url += f'&$skip={skip}'

        resp = requests.get(url, headers=headers)
        print(url)
        print(resp.status_code)

        search_results = resp.json()
        agg_search_results.append(search_results)
        print("Results Found: {}, Results Returned: {}".format(search_results['@odata.count'], len(search_results['value'])))
        return agg_search_results
agg_search_results = get_agg_search_results()
#agg_search_results

https://servicehealth-summaryparser.search.windows.net/indexes/servicehealthfhl-index1/docs?api-version=2021-04-30-Preview&search=what are the Authentication issues?&select=*&$top=5&queryLanguage=en-us&queryType=semantic&semanticConfiguration=servicehealthfhl-semantic-config&$count=true&speller=lexicon&answers=extractive|count-3&captions=extractive|highlight-false&$skip=0
200
Results Found: 7, Results Returned: 5


filter and order search results by score:

In [78]:
# from the above responses more filtering is possible simmilar to 

#display(HTML('<h4>Top Answers</h4>'))

def sort_and_order_content(agg_search_results):
    for search_results in agg_search_results:
        for result in search_results['@search.answers']:
            print(result['score'])
            if result['score'] > 0.5: # Show answers that are at least 50% of the max possible score=1
                print("got here")
                display(HTML('<h5>' + 'Answer - score: ' + str(round(result['score'],2)) + '</h5>'))
                display(HTML(result['text']))
                
    #print("\n\n")
    #display(HTML('<h4>Top Results</h4>'))

    content = dict()
    ordered_content = OrderedDict()


    for search_results in agg_search_results:
        for result in search_results['value']:
            if result['@search.rerankerScore'] > 0.5: # Filter results that are at least 12.5% of the max possible score=4
                content[result['id']]={
                                        "title": result['title'],
                                        "chunks": result['pages'],
                                        "language": result['language'], 
                                        "caption": result['@search.captions'][0]['text'],
                                        "score": result['@search.rerankerScore'],
                                        #"name": result['metadata_storage_name'], 
                                        #"location": result['metadata_storage_path']                  
                                    }
                
    #print(json.dumps(content, indent=4))
        
    #After results have been filtered we will Sort and add them as an Ordered list\n",
    for id in sorted(content, key= lambda x: content[x]["score"], reverse=True):
        ordered_content[id] = content[id]
        #url = ordered_content[id]['location'] + DATASOURCE_SAS_TOKEN
        title = str(ordered_content[id]['title']) if (ordered_content[id]['title']) else ordered_content[id]['name']
        score = str(round(ordered_content[id]['score'],2))
        #display(HTML('<h5><a href="'+ url + '">' + title + '</a> - score: '+ score + '</h5>'))
        print(f"${id} - ${title} - ${score}")
        #display(HTML(ordered_content[id]['caption']))

    return ordered_content

ordered_content = sort_and_order_content(agg_search_results)
#print(json.dumps(ordered_content, indent=4))
docs  = create_langchain_documents(ordered_content)

$FLX7-VD8 - $RCA - Azure Multi-Factor Authentication - MFA Pull Down Refresh Issues - $1.0
$M0L-VC0 - $PIR/ RCA - Azure Multi-Factor Authentication - Issues with Push Notifications - $0.95
$RKWL-T98 - $PIR/RCA - Azure Multi-Factor Authentication - Issues using Voice for Multi-Factor Authentication (MFA) in Philippines - $0.91
$BL8Y-DT8 - $Azure Multi-Factor Authentication (MFA) - Applying Mitigation - $0.76
$FKPH-7Z8 - $PIR - Azure Multi-Factor Authentication - $0.65


In [79]:



def get_docs_wrapper(skip):
    agg_search_results = get_agg_search_results()
    ordered_content = sort_and_order_content(agg_search_results)
    docs  = create_langchain_documents(ordered_content)
    return docs
    
    
        
print("Number of chunks for chat gpt to use:",len(docs))

Number of chunks for chat gpt to use: 5


In [80]:


# Calculate number of tokens of our docs
if(len(docs)>0):
    tokens_limit = model_tokens_limit(MODEL) 
    num_tokens = num_tokens_from_docs(docs) 
    print("Custom token limit for", MODEL, ":", tokens_limit)
    print("Combined docs tokens count:",num_tokens)
        
else:
    print("NO RESULTS FROM AZURE SEARCH")

docs

Custom token limit for gpt-35-turbo : 3000
Combined docs tokens count: 1705


[Document(page_content='What happened?\nYou were impacted by a platform issue that caused some of our Multi-Factor Authentication (MFA) customers to experience issues when attempting a pull-down refresh using the Microsoft Authenticator app on Android devices, between 19:19 UTC on 6 Mar 2023 and 22:20 UTC on 21 March 2023. While this issue impacted only one scenario where customers used a pull-down refresh instead of interacting with the push notification, unfortunately it impacted one or more of your Azure subscriptions. That’s why we’re providing you with this Post Incident Review (PIR) – to summarize what went wrong, how we responded, and the steps Microsoft is taking to learn from this and improve.\nWhat went wrong and why?\nThe Microsoft Authenticator app has support for a client-initiated refresh to allow customers to poll for a pending notification from the service. A recent service update contained a new feature that exposed a code issue when refreshing the Microsoft Authentica

The vector search:

In [81]:
 
def get_chain_type_and_top_docs(tokens_limit,num_tokens,docs):    
    print(num_tokens)
    if num_tokens ==0 or num_tokens > tokens_limit: # need to do a vector search in these cases
        if num_tokens==0:
            pass
        # Select the Embedder model
        if len(docs) < 50:
            # OpenAI models are accurate but slower, they also only (for now) accept one text at a time (chunk_size)
            embedder = OpenAIEmbeddings(deployment="text-embedding-ada-002", chunk_size=1) 
        else:
            # Bert based models are faster (3x-10x) but not as great in accuracy as OpenAI models
            # Since this repo supports Multiple languages we need to use a multilingual model. 
            # But if English only is the requirement, use "multi-qa-MiniLM-L6-cos-v1"
            # The fastest english model is "all-MiniLM-L12-v2"
            embedder = HuggingFaceEmbeddings(model_name = 'distiluse-base-multilingual-cased-v2')
        
        print(embedder)
        
        # Create our in-memory vector database index from the chunks given by Azure Search.
        # We are using FAISS. https://ai.facebook.com/tools/faiss/
        db = FAISS.from_documents(docs, embedder)
        top_docs = db.similarity_search(QUESTION, k=4)  # Return the top 4 documents
        
        # Now we need to recalculate the tokens count of the top results from similarity vector search
        # in order to select the chain type: stuff (all chunks in one prompt) or 
        # map_reduce (multiple calls to the LLM to summarize/reduce the chunks and then combine them)
        
        num_tokens = num_tokens_from_docs(top_docs)
        print("Token count after similarity search:", num_tokens)
        chain_type = "map_reduce" if num_tokens > tokens_limit else "stuff"
        
    else:
        # if total tokens is less than our limit, we don't need to vectorize and do similarity search
        top_docs = docs
        chain_type = "stuff"
    
    
    return top_docs,chain_type

 
top_docs,chain_type = get_chain_type_and_top_docs(tokens_limit,num_tokens,docs)
    
print("Chain Type selected:", chain_type)


1705
Chain Type selected: stuff


#### Search is complete time to Summarize the data:

In [82]:
COMBINE_QUESTION_PROMPT_TEMPLATE = """Use the following portion of a long document to see if any of the text is relevant to answer the question. 
Return any relevant text in {language}.
{context}
Question: {question}
Relevant text, if any, in {language}:"""

COMBINE_QUESTION_PROMPT = PromptTemplate(
    template=COMBINE_QUESTION_PROMPT_TEMPLATE, input_variables=["context", "question", "language"]
)


COMBINE_PROMPT_TEMPLATE = """
These are examples of how you must provide the answer:
--> Beginning of examples
=========
QUESTION: Which state/country's law governs the interpretation of the contract?
=========
Content: This Agreement is governed by English law and the parties submit to the exclusive jurisdiction of the English courts in  relation to any dispute (contractual or non-contractual) concerning this Agreement save that either party may apply to any court for an  injunction or other relief to protect its Intellectual Property Rights.
Source: https://xxx.com/article1.pdf
Content: No Waiver. Failure or delay in exercising any right or remedy under this Agreement shall not constitute a waiver of such (or any other)  right or remedy.\n\n11.7 Severability. The invalidity, illegality or unenforceability of any term (or part of a term) of this Agreement shall not affect the continuation  in force of the remainder of the term (if any) and this Agreement.\n\n11.8 No Agency. Except as expressly stated otherwise, nothing in this Agreement shall create an agency, partnership or joint venture of any  kind between the parties.\n\n11.9 No Third-Party Beneficiaries.
Source: https://yyyy.com/article2.html
Content: (b) if Google believes, in good faith, that the Distributor has violated or caused Google to violate any Anti-Bribery Laws (as  defined in Clause 8.5) or that such a violation is reasonably likely to occur,
Source: https://yyyy.com/article3.csv
Content: The terms of this Agreement shall be subject to the laws of Manchester, England, and any disputes arising from or relating to this Agreement shall be exclusively resolved by the courts of that state, except where either party may seek an injunction or other legal remedy to safeguard their Intellectual Property Rights.
Source: https://ppp.com/article4.pdf
=========
FINAL ANSWER IN English: This Agreement is governed by English law, specifically the laws of Manchester, England.
SOURCES: https://xxx.com/article1.pdf, https://ppp.com/article4.pdf
=========
QUESTION: What did the president say about Michael Jackson?
=========
Content: Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  \n\nLast year COVID-19 kept us apart. This year we are finally together again. \n\nTonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. \n\nWith a duty to one another to the American people to the Constitution. \n\nAnd with an unwavering resolve that freedom will always triumph over tyranny. \n\nSix days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. \n\nHe thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. \n\nHe met the Ukrainian people. \n\nFrom President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world. \n\nGroups of citizens blocking tanks with their bodies. Everyone from students to retirees teachers turned soldiers defending their homeland.
Source: https://fff.com/article23.pdf
Content: And we won’t stop. \n\nWe have lost so much to COVID-19. Time with one another. And worst of all, so much loss of life. \n\nLet’s use this moment to reset. Let’s stop looking at COVID-19 as a partisan dividing line and see it for what it is: A God-awful disease.  \n\nLet’s stop seeing each other as enemies, and start seeing each other for who we really are: Fellow Americans.  \n\nWe can’t change how divided we’ve been. But we can change how we move forward—on COVID-19 and other issues we must face together. \n\nI recently visited the New York City Police Department days after the funerals of Officer Wilbert Mora and his partner, Officer Jason Rivera. \n\nThey were responding to a 9-1-1 call when a man shot and killed them with a stolen gun. \n\nOfficer Mora was 27 years old. \n\nOfficer Rivera was 22. \n\nBoth Dominican Americans who’d grown up on the same streets they later chose to patrol as police officers. \n\nI spoke with their families and told them that we are forever in debt for their sacrifice, and we will carry on their mission to restore the trust and safety every community deserves.
Source: https://jjj.com/article56.pdf
Content: And a proud Ukrainian people, who have known 30 years  of independence, have repeatedly shown that they will not tolerate anyone who tries to take their country backwards.  \n\nTo all Americans, I will be honest with you, as I’ve always promised. A Russian dictator, invading a foreign country, has costs around the world. \n\nAnd I’m taking robust action to make sure the pain of our sanctions  is targeted at Russia’s economy. And I will use every tool at our disposal to protect American businesses and consumers. \n\nTonight, I can announce that the United States has worked with 30 other countries to release 60 Million barrels of oil from reserves around the world.  \n\nAmerica will lead that effort, releasing 30 Million barrels from our own Strategic Petroleum Reserve. And we stand ready to do more if necessary, unified with our allies.  \n\nThese steps will help blunt gas prices here at home. And I know the news about what’s happening can seem alarming. \n\nBut I want you to know that we are going to be okay.
Source: https://vvv.com/article145.pdf
Content: More support for patients and families. \n\nTo get there, I call on Congress to fund ARPA-H, the Advanced Research Projects Agency for Health. \n\nIt’s based on DARPA—the Defense Department project that led to the Internet, GPS, and so much more.  \n\nARPA-H will have a singular purpose—to drive breakthroughs in cancer, Alzheimer’s, diabetes, and more. \n\nA unity agenda for the nation. \n\nWe can do this. \n\nMy fellow Americans—tonight , we have gathered in a sacred space—the citadel of our democracy. \n\nIn this Capitol, generation after generation, Americans have debated great questions amid great strife, and have done great things. \n\nWe have fought for freedom, expanded liberty, defeated totalitarianism and terror. \n\nAnd built the strongest, freest, and most prosperous nation the world has ever known. \n\nNow is the hour. \n\nOur moment of responsibility. \n\nOur test of resolve and conscience, of history itself. \n\nIt is in this moment that our character is formed. Our purpose is found. Our future is forged. \n\nWell I know this nation.
Source: https://uuu.com/article15.pdf
=========
FINAL ANSWER IN English: The president did not mention Michael Jackson.
SOURCES: N/A
<-- End of examples
Given the following extracted parts of a long document and a question, create a final answer with references ("SOURCES"). 
If you don't know the answer, just say that you don't know. Don't try to make up an answer.
ALWAYS return a "SOURCES" part in your answer.
Instead of using the word "Document" use "Tracking ID"
Respond in {language}.
=========
QUESTION: {question}
=========
{summaries}
=========
FINAL ANSWER IN {language}:"""


COMBINE_PROMPT = PromptTemplate(
    template=COMBINE_PROMPT_TEMPLATE, input_variables=["summaries", "question", "language"]
)

In [83]:
# Set the ENV variables that Langchain needs to connect to Azure OpenAI
os.environ["OPENAI_API_BASE"] = os.environ["AZURE_OPENAI_ENDPOINT"] = AZURE_OPENAI_ENDPOINT
os.environ["OPENAI_API_KEY"] = os.environ["AZURE_OPENAI_API_KEY"] = AZURE_OPENAI_API_KEY
os.environ["OPENAI_API_VERSION"] = os.environ["AZURE_OPENAI_API_VERSION"] = AZURE_OPENAI_API_VERSION
os.environ["OPENAI_API_TYPE"] = "azure"   

# Create our LLM model
# Make sure you have the deployment named "gpt-35-turbo" for the model "gpt-35-turbo (0301)". 
# Use "gpt-4" if you have it available.

llm2 = AzureChatOpenAI(deployment_name=MODEL, temperature=0, max_tokens=500)

if chain_type == "stuff":
    chain = load_qa_with_sources_chain(llm2, chain_type=chain_type, 
                                       prompt=COMBINE_PROMPT)
elif chain_type == "map_reduce":
    chain = load_qa_with_sources_chain(llm2, chain_type=chain_type, 
                                       question_prompt=COMBINE_QUESTION_PROMPT,
                                       combine_prompt=COMBINE_PROMPT,
                                       return_intermediate_steps=True)



In [84]:
response = chain({"input_documents": top_docs, "question": QUESTION, "language": "English"})
answer = response['output_text']
print(response)
print(answer)


Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Requests to the Creates a completion for the chat message Operation under Azure OpenAI API version 2023-03-15-preview have exceeded token rate limit of your current OpenAI S0 pricing tier. Please retry after 2 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit..


{'input_documents': [Document(page_content='What happened?\nYou were impacted by a platform issue that caused some of our Multi-Factor Authentication (MFA) customers to experience issues when attempting a pull-down refresh using the Microsoft Authenticator app on Android devices, between 19:19 UTC on 6 Mar 2023 and 22:20 UTC on 21 March 2023. While this issue impacted only one scenario where customers used a pull-down refresh instead of interacting with the push notification, unfortunately it impacted one or more of your Azure subscriptions. That’s why we’re providing you with this Post Incident Review (PIR) – to summarize what went wrong, how we responded, and the steps Microsoft is taking to learn from this and improve.\nWhat went wrong and why?\nThe Microsoft Authenticator app has support for a client-initiated refresh to allow customers to poll for a pending notification from the service. A recent service update contained a new feature that exposed a code issue when refreshing the 

In [85]:
display(HTML('<h4>Azure OpenAI ChatGPT Answer:</h4>'))
display(HTML(answer.split("SOURCES:")[0]))



In [86]:

#we get the sources and can add urls too(if wanted)

sources_list = answer.split("SOURCES:")[1].replace(" ","").split(",")

sources_html = '<u>Sources</u>: '
display(HTML(sources_html))
for index, value in enumerate(sources_list):
    print(value)
    #url = value + DATASOURCE_SAS_TOKEN 
    #sources_html +='<sup><a href="'+ url + '">[' + str(index+1) + ']</a></sup>'
    



TrackingIDFLX7-VD8
M0L-VC0
RKWL-T98
BL8Y-DT8
FKPH-7Z8.
